<a href="https://colab.research.google.com/github/guptapallavi30/ScalableML_1/blob/exploration/TextHypothesisClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

# Upload your Kaggle API key file to "/content/kaggle/kaggle.json"
# Replace "/content/kaggle/kaggle.json" with the path where you placed your Kaggle API key file in Colab.
!mkdir -p ~/.kaggle
!cp "../content/kaggle/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
# Download the dataset
import pandas as pd

# Replace "narendrageek/my-dear-watson-translated-text" with the dataset name you want to use.
!kaggle datasets download -d narendrageek/my-dear-watson-translated-text -p /content

# Unzip the downloaded dataset (if necessary).
import zipfile
with zipfile.ZipFile("/content/my-dear-watson-translated-text.zip", "r") as zip_ref:
    zip_ref.extractall("/content")

# Read the dataset into a pandas DataFrame.
df = pd.read_csv("/content/translated_train_dataset.csv")

# Read the dataset into a pandas DataFrame.
dftest = pd.read_csv("/content/translated_test_dataset.csv")

  0% 0.00/3.10M [00:00<?, ?B/s]
100% 3.10M/3.10M [00:00<00:00, 138MB/s]


In [3]:
df.head()

,id,premise,hypothesis,lang_abv,language,label,premise_translated,hypothesis_translated
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0,and these comments were considered in formulat...,The rules developed in the interim were put to...
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0,Little things like these make a huge differenc...,I was trying to accomplish something.
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0,you know they can't really defend themselves l...,They can't defend themselves because of their ...
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1,In role playing as well The opportunity to exp...,Children can see how different ethnic groups are.


In [4]:
dftest.head()

,id,premise,hypothesis,lang_abv,language,label,premise_translated,hypothesis_translated
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0,and these comments were considered in formulat...,The rules developed in the interim were put to...
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0,Little things like these make a huge differenc...,I was trying to accomplish something.
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0,you know they can't really defend themselves l...,They can't defend themselves because of their ...
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1,In role playing as well The opportunity to exp...,Children can see how different ethnic groups are.


In [5]:
df_train_shape = df.shape
df_test_shape = dftest.shape
print(df_train_shape, df_test_shape)

(12120, 8) (12120, 8)


In [6]:
# Combine train and test
df_all = [df, dftest]
df_all_frames = [df, dftest]

df_all = pd.concat(df_all_frames)

In [7]:
df_all.shape

(24240, 8)

In [28]:
import pandas as pd

# Check for duplicate rows
duplicates = df_all[df_all.duplicated()]

# If 'duplicates' DataFrame is empty, there are no duplicate rows
if duplicates.empty:
    print("There are no duplicate rows in the dataset.")
else:
    print("Duplicate rows exist in the dataset:")
    print(duplicates.shape)

Duplicate rows exist in the dataset:
(12009, 11)


In [8]:
# List of column names for which you want to get unique values and value counts.
selected_columns = ['language', 'label']

for column in selected_columns:
    value_counts = df_all[column]
    null_values = df_all[column].isnull().sum()
    print(f"Column: {column}")
    print("Null Values:", null_values)
    print("Value Counts:")
    print(value_counts)
    print("="*30)

Column: language
Null Values: 0
Value Counts:
0        English
1        English
2         French
3        English
4           Thai
          ...   
12115    English
12116    English
12117    English
12118    English
12119    English
Name: language, Length: 24240, dtype: object
Column: label
Null Values: 0
Value Counts:
0        0
1        2
2        0
3        0
4        1
        ..
12115    2
12116    0
12117    2
12118    2
12119    0
Name: label, Length: 24240, dtype: int64


FROM [DATASET APPROACH SECTION](https://www.kaggle.com/datasets/narendrageek/my-dear-watson-translated-text):

"*The dataset has multiple languages. Hence, I have translated Non-English sentences to English sentences using the Google Transcription python library*"

English texts heavily dominate the language section. For the purpose of simplifying the model, I will choose the English-translated columns as input for the model.

In [9]:
!pip install langdetect
from langdetect import detect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=2aecd861547bbdf769e9a6d5963b972a41b8c6053ed9e179274857e8e33a2de7
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [10]:
# Verify all entries in 'premise_translated' and 'hypothesis_translated' are in English
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

# Applying the 'is_english' function to the columns
df_all['is_english_premise_t'] = df_all.premise_translated.apply(is_english)
df_all['is_english_hypothesis_t'] = df_all.hypothesis_translated.apply(is_english)

In [11]:
all_entries_in_english_p = (df_all.is_english_premise_t == False).sum()
all_entries_in_english_h = (df_all.is_english_hypothesis_t == False).sum()

print(df_all.shape[0])
print("All entries in 'premise_translated' are in English:", all_entries_in_english_p)
print("All entries in 'hypothesis_translated' are in English:", all_entries_in_english_h)

24240
All entries in 'premise_translated' are in English: 4623
All entries in 'hypothesis_translated' are in English: 4670


In [12]:
print((df_all.lang_abv[(df_all.is_english_premise_t == False) & (df_all.lang_abv != 'en')]).value_counts())
print('-------')
print((df_all.lang_abv[(df_all.is_english_hypothesis_t == False) & (df_all.lang_abv != 'en')]).value_counts())

#.iloc[0]

zh    327
vi    324
fr    316
sw    316
ur    312
ru    312
ar    301
th    299
hi    297
tr    297
el    292
de    273
bg    268
es    254
Name: lang_abv, dtype: int64
-------
zh    355
ar    334
ru    325
ur    319
th    318
vi    313
hi    310
fr    306
tr    305
el    296
sw    289
de    282
es    279
bg    267
Name: lang_abv, dtype: int64


In [13]:
!pip install googletrans==3.1.0a0
# For all entries that are not in english, convert them to English ourselves
from googletrans import Translator

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.6 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=e2dcf5e7173b16a82837d90486400cf59a0e2984a60562939317dcbe2713b2cf
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling cha

In [14]:
def translate_to_english(sentence, source_language='auto', target_language='en'):
    translator = Translator()
    translated_text = translator.translate(sentence, src=source_language, dest=target_language)
    return translated_text.text

In [15]:
# Filtering rows based on the conditions and applying the function
mask = (df_all.is_english_premise_t == False) & (df_all.lang_abv != 'en')
df_all.loc[mask, 'premise_translate_modified'] = df_all.loc[mask, 'premise'].apply(translate_to_english)

print(df_all.loc[mask, 'premise_translate_modified'])

186      vahi baat New York Times ke liye nahi kaha ja ...
292      Yes I actually had older old singers or older ...
325      Hundreds of excursionists who sail to both isl...
341      Forming part of the city walls, the gate was p...
363      Despite a two-year investigation, the FBI was ...
                               ...                        
12093    Although there is an ongoing kindness of donat...
12099    But my job was to put parachutes on it and lif...
12106    C-R functions can also be estimated with or wi...
12111    -design, draw and hand sew all those gorgeous ...
12112    We didn't know what U2 was and nobody knew any...
Name: premise_translate_modified, Length: 4188, dtype: object


In [16]:
# verifying how many passed test
passed_test = df_all.loc[mask, 'premise_translate_modified'].apply(is_english)
remaining = (passed_test == False).sum()
remaining

111

In [ ]:
# Translate the non-English statements with ChatGPT

In [26]:
# get list of all values to translate
to_translate = []
for i in df_all.loc[mask, 'premise_translate_modified'][passed_test == False]:
  to_translate.append(i)
to_translate

['vahi baat New York Times ke liye nahi kaha ja kakata hai. Cocaine use of drugs, Times ne Bush ko eemanadaar hone kee salaah dee aur kaha ki desh ko aapana upaay karane den.',
 'Yes I actually had older old singers or older older sisters singers',
 'NEW ARRIVALS EVERY GIFT MAKES A DIFFERENCE!',
 'If you get upset you will get dizzy.',
 'A postcode can serve multiple routes.',
 'yes actually i have older singers like this or older singers older sisters',
 "And I was like, I'm almost done.",
 '.. spiritual guidance and encouragement.',
 "it's like comparing savings",
 'A showdown, yeah!',
 'Un talabon mese ek hai jo paridarshakon ke dwara ek kachhue ke head se chalalane kee aasha mein feke gaye sikkon se bhara hai, jo achchhe bhaagy ko umid mein ki umid karne jata hai.',
 "Yes, it's just impossible, isn't it?",
 'Ticket sales and subscriptions cannot fund our entire run.',
 'Email, haalaaki, yah pratibimbit hai ki vah warnings hai ki vah warnings ki evam soochana sahabhajan karane ka ba

"That thing is not said for the New York Times. The use of cocaine, Times advised Bush to be honest and said, 'Let the country decide its own remedy.'"
"Yes, I actually had older sisters who were singers."
"NEW ARRIVALS EVERY GIFT MAKES A DIFFERENCE!"
"If you get upset, you will get dizzy."
"A postcode can serve multiple routes."
"Yes, actually I have older singers like this or older sisters."
"And I was like, I'm almost done."
".. spiritual guidance and encouragement."
"It's like comparing savings."
"A showdown, yeah!"
"There is a pond among the ponds that is filled with coins thrown in the hope of moving from the head of a turtle by the spectators, hoping for good luck."
"Yes, it's just impossible, isn't it?"
"Ticket sales and subscriptions cannot fund our entire run."
"Email, however, indicates that she was actively concealing information from an authorized agent of the commission by secretly channeling information through an intermediary."
"BAHARL That man enters."
"That thing is not said for the New York Times. The use of cocaine, Times advised Bush to be honest and said, 'Let the country decide its own remedy.'"
"How do you dance aerobics?"
"Let's say break, steak, but bleak and streak."
"Four... four... siku oh him."
"Would I order a set for myself?"
"Ticket sales and subscriptions cannot fund our entire run."
"Please do not slide on our donor list."
"You won't get a worse answer."
"If you get upset, you will get dizzy."
"Say break (break), steak (steak), but bleak (glare), and streak (streak)."
"Said my wife, raising an eyebrow."
"My name is Dwyane--Lord Julian Dwyane."
"My name is Wade, Lord Julian Wade."
"I... I'm not dreaming."
".. spiritual guidance and encouragement."
"Well, not even tell me lately."
"Yeah, ha ha, yeah, okay, it's Fresno, ha ha!"
"Believe me, I am very grateful."
"It only seems to be getting worse."
"Trust me, I appreciate it."
"Revenge."
"Stepping on some strong big toes."
"Name, like a sad girl."
".. spirit of guidance and encouragement."
"I don't have time to do it all."
"In this context, despite the publication of 1,500,000 entries in the United States Omni Gazetteer, one can only see a significant event in the first step."
"Yeah, it just doesn't seem possible, does it?"
"Names, like Sad Girl."
"Sure, sure, don't go all out and."
"How would you do aerobics?"
"He missed."
"How do you do aerobics?"
"(sad) No, no, I don't want you to die!"
".. Make our society better."
"Yes, yes, okay, goodbye."
"Would I order a set for myself?"
"Levasseur? He smiled a little."
"He is a blood drinker."
"Yes, I actually had older singers like that or even older singers and older sisters."
"Government/juridical batta, begar, chaprasi, dakoit, dacoity, dhan, dharna, kotwal, kotwali, panchayat, pottah, sabha."
"Levasseur? He smiled a little."

In [21]:
for i in df_all.loc[mask, 'id'][passed_test == False]:
  print(i + " " + df_all.premise_translate_modified[df_all.id == i])

186    6960d6ed8a vahi baat New York Times ke liye na...
186    6960d6ed8a vahi baat New York Times ke liye na...
Name: premise_translate_modified, dtype: object
292    24e9a5e06d Yes I actually had older old singer...
292    24e9a5e06d Yes I actually had older old singer...
Name: premise_translate_modified, dtype: object
405    64939096f7 NEW ARRIVALS EVERY GIFT MAKES A DIF...
405    64939096f7 NEW ARRIVALS EVERY GIFT MAKES A DIF...
Name: premise_translate_modified, dtype: object
619    e76f0873f4 If you get upset you will get dizzy.
619    e76f0873f4 If you get upset you will get dizzy.
Name: premise_translate_modified, dtype: object
670    19b0abb2c4 A postcode can serve multiple routes.
670    19b0abb2c4 A postcode can serve multiple routes.
Name: premise_translate_modified, dtype: object
1457    6eff086099 yes actually i have older singers l...
1457    6eff086099 yes actually i have older singers l...
Name: premise_translate_modified, dtype: object
1735    903ea31eaa And I was lik

In [ ]:
for i in df_all.loc[mask, 'id'][passed_test == False]:
  print(i + " " + df_all.premise_translate_modified[df_all.id == i])

In [ ]:
# translate premise
num_iters = [(df_all.is_english_premise_t == False).sum(), (df_all.is_english_hypothesis_t == False).sum()]
translating_premise = True

# Applying the 'is_english' function to the 'text_column'
# df_all['test_translating_p'] = df_all.premise.apply(translate_to_english)
# df_all['test_translating_h'] = df_all.hypothesis.apply(translate_to_english)

# for num_iter in num_iters:
for i in (df_all.is_english_premise_t == False):
  # print(i)
  # Example sentence in a foreign language (French)
  foreign_sentence = df_all.premise[(df_all.is_english_premise_t == False) & (df_all.lang_abv != 'en')].iloc[i]

  # Translate the foreign sentence to English
  english_translation = translate_to_english(foreign_sentence)

  print("Foreign Sentence:", foreign_sentence)
  print("English Translation:", english_translation)

TypeError: ignored

In [ ]:
# Applying the 'is_english' function to the 'text_column'
df_all['is_english_premise_t'] = df_all.premise_translated.apply(is_english)
df_all['is_english_hypothesis_t'] = df_all.hypothesis_translated.apply(is_english)

----------

In [ ]:
selected_columns = ['premise_translated','hypothesis_translated','label']

# Reduce the DataFrame to the specific columns using the indexing operator [].
reduced_df_all = df_all[selected_columns]

In [ ]:
reduced_df_all.head()

,premise_translated,hypothesis_translated,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2
2,Little things like these make a huge differenc...,I was trying to accomplish something.,0
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0
4,In role playing as well The opportunity to exp...,Children can see how different ethnic groups are.,1


In [ ]:
# Data Processing
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Step 1: Tokenization
sentences = [reduced_df_all.premise_translated, reduced_df_all.hypothesis_translated]
sentences = pd.concat(sentences)
print(type(sentences), sentences)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

<class 'pandas.core.series.Series'> 0        and these comments were considered in formulat...
1        These are issues that we wrestle with in pract...
2        Little things like these make a huge differenc...
3        you know they can't really defend themselves l...
4        In role playing as well The opportunity to exp...
                               ...                        
12115    All studies have the same amount of uncertaint...
12116    But there are two kinds of the pleasure of doi...
12117                     It cannot be moved, now or ever.
12118         The model temple complex is at the east end.
12119        Ataturk was the father of the Turkish nation.
Length: 48480, dtype: object


In [ ]:
tokenizer.index_word.get(len(tokenizer.index_word))

'เด็กที่ถูกทารุณและถูกทอดทิ้งส่วนใหญ่เป็นคนกลุ่มน้อย'